In [1]:
import os
os.path

<module 'posixpath' from '/home/tylerf/tensorflow/lib/python3.5/posixpath.py'>

In [4]:
import logging
import os
import pickle
import pandas
from multiprocessing import Pool
import math
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import keras
import csv

Using TensorFlow backend.


In [130]:
class linear():
    """
    Linear Model Class: A SLR model class
    param: theta - slope for 
    param: sigma - Standard deviation
    param: n - sample size
    function: simulate() - simulate model data
    function: plot() - plot model data with true
    TODO: make lin_x and lin_y attributes?
    """

    def __init__(self, theta=2, sigma=10, x_min=0, x_max=20):
        self.sigma = sigma
        self.theta = theta
        self.x_min = x_min
        self.x_max = x_max
        self.x_dim = 1
        self.y_dim = 1

    def simulate(self, n):
        x = np.sort(np.random.uniform(self.x_min, self.x_max, n))
        y = self.theta * x + np.random.normal(0, self.sigma, n)
        return pandas.DataFrame({'x': x, 'y': y})

    def plot(self, sample_space):
        lin_x = np.linspace(self.x_min, self.x_max, 100)
        lin_y = self.theta * lin_x
        matplotlib.pyplot.scatter(sample_space['x'], sample_space['y'])
        matplotlib.pyplot.plot(lin_x, lin_y)
        matplotlib.pyplot.show()


class nonlinear():
    """
    Linear Model Class: A SLR model class
    param: theta - slope for
    param: sigma - Standard deviation
    param: n - sample size
    simulate - function
    TODO: make lin_x and lin_y attributes?
    """

    def __init__(self, n=1000, sigma=10, theta_1=80, theta_2=71, theta_3=-10, theta_4=2.4, x_min=0, x_max=90):
        self.n = n
        self.sigma = sigma
        self.theta_1 = 80
        self.theta_2 = 71
        self.theta_3 = -10
        self.theta_4 = 2.4
        self.x_min = 0
        self.x_max = 90
        #self.df = self.simulate()

    def simulate(self,sample_size):
        x = sorted(np.random.uniform(self.x_min, self.x_max, sample_size))
        y = self.theta_1 - self.theta_2 * np.exp(
        -np.exp(self.theta_3 + self.theta_4 * np.log(x))) + np.random.normal(0, self.sigma, sample_size)
        return pandas.DataFrame({'x': x, 'y': y})

    def theta(self, x=45):
        return(self.theta_1 - self.theta_2 * np.exp(
        -np.exp(self.theta_3 + self.theta_4 * np.log(x))))

    def plot(self):
        lin_x = np.linspace(self.x_min, self.x_max, 100)
        lin_y = self.theta_1 - self.theta_2 * np.exp(-np.exp(self.theta_3 + self.theta_4 * np.log(lin_x)))
        matplotlib.pyplot.scatter(self.df['x'], self.df['y'])
        matplotlib.pyplot.plot(lin_x, lin_y)
        matplotlib.pyplot.show()


class mars():
    def __init__(self, sample_size=1000):
        self.x_min = 0
        self.x_max = 1
        self.x_dim = 5
        self.sample_size = sample_size
        self.y_dim = 1
        

    def simulate(self):
        x = np.random.uniform(self.x_min, self.x_max, size=(self.sample_size, 5))     
        y = 10*np.sin(np.pi*x[:,0]*x[:,1]) - 20*(x[:,2]-0.05)**2 + 10*x[:,3] + 5*x[:,4] + np.random.normal(0, 10, self.sample_size)
        return (x,y)


In [126]:
# s=np.random.uniform(0, 1, size = (3,5))
print(s)
s[:,1]

[[0.37070824 0.813501   0.75511687 0.25120105 0.0779309 ]
 [0.72681793 0.41334843 0.26066188 0.20718389 0.50728908]
 [0.59961194 0.23996353 0.66394911 0.05422526 0.30008556]]


array([0.813501  , 0.41334843, 0.23996353])

In [137]:
trainX, trainY= mars(100000).simulate()
trainX.shape

x_pred = np.array([.5,.5,.5,.5,.5])
x_pred.shape

(5,)

In [161]:
debug = False
num_lay_nodes = 12
lr = .01
epochs = 3000
keras_verbose = 1
trainX, trainY = mars(10000).simulate()

def build_mars_estimator():  

    nn1 = keras.Sequential()
    # This is worth about 9% if we cut neurons down to 25
    nn1.add(keras.layers.Dense(num_lay_nodes, kernel_initializer="uniform", 
                               activation='sigmoid', input_dim=5))
    nn1.add(keras.layers.Dense(units=1, activation='linear'))
    # training
    # sgd = keras.optimizers.SGD(lr=0.0001);
    adam = keras.optimizers.Adam(lr=lr)
    # model.compile(loss='mse', optimizer=sgd)
    nn1.compile(loss='mse', optimizer=adam)
    return(nn1)


    
def estimate(nn,trainX, trainY):    
    
    
    ##This isn't strictly necessary but maintaisn the same intialiation 
    ## parameterization for each neural network. 
    Wsave = nn.get_weights()
    nn.fit(trainX, trainY, epochs=epochs,verbose=keras_verbose)

    # predictions
    # Linear predict 10 
    #x_pred = np.arange(10, 11, 1, dtype='float32').reshape(-1, 1)
   
    x_pred = np.array([.5,.5,.5,.5,.5])
    predictions = nn.predict(x_pred.reshape((1,5)))
    nn.set_weights(Wsave)
    
    if debug:
        plt.scatter(simulated_data['x'], simulated_data['y'], edgecolors='g')
        plt.plot(x_pred, predictions, 'r')
        plt.legend(['Predicted Y', 'Actual Y'])
        plt.show()
    
    return(predictions)

estimator = build_mars_estimator()
p = list()
start_time = datetime.now()
for _ in range(1):    
    p.append(estimate(estimator, trainX, trainY))

print("This took {} seconds to run".format(datetime.now()-start_time))
print("Mean {} Var {}".format(np.mean(p),np.var(p)))




Epoch 1/3000
10000/10000 [==============================] - 1s 70us/step - loss: 145.2676
Epoch 2/3000
10000/10000 [==============================] - 0s 48us/step - loss: 114.9400
Epoch 3/3000
10000/10000 [==============================] - 0s 50us/step - loss: 109.5012
Epoch 4/3000
10000/10000 [==============================] - 0s 50us/step - loss: 108.6767
Epoch 5/3000
10000/10000 [==============================] - 0s 48us/step - loss: 108.4974
Epoch 6/3000
10000/10000 [==============================] - 0s 49us/step - loss: 108.4600
Epoch 7/3000
10000/10000 [==============================] - 1s 50us/step - loss: 108.3904
Epoch 8/3000
10000/10000 [==============================] - 0s 50us/step - loss: 108.2596
Epoch 9/3000
10000/10000 [==============================] - 1s 52us/step - loss: 108.1386
Epoch 10/3000
10000/10000 [==============================] - 1s 51us/step - loss: 108.1967
Epoch 11/3000
10000/10000 [==============================] - 1s 54us/step - loss: 108.1149
Epoch 12

10000/10000 [==============================] - 0s 47us/step - loss: 103.2764
Epoch 92/3000
10000/10000 [==============================] - 0s 49us/step - loss: 103.3733
Epoch 93/3000
10000/10000 [==============================] - 0s 47us/step - loss: 103.2824
Epoch 94/3000
10000/10000 [==============================] - 0s 47us/step - loss: 103.0668
Epoch 95/3000
10000/10000 [==============================] - 0s 48us/step - loss: 103.0921
Epoch 96/3000
10000/10000 [==============================] - 0s 47us/step - loss: 103.1271
Epoch 97/3000
10000/10000 [==============================] - 0s 47us/step - loss: 103.1251
Epoch 98/3000
10000/10000 [==============================] - 0s 46us/step - loss: 103.0877
Epoch 99/3000
10000/10000 [==============================] - 0s 47us/step - loss: 103.1143
Epoch 100/3000
10000/10000 [==============================] - 0s 47us/step - loss: 103.0120
Epoch 101/3000
10000/10000 [==============================] - 0s 49us/step - loss: 102.9982
Epoch 102/3

10000/10000 [==============================] - 0s 48us/step - loss: 101.7993
Epoch 181/3000
10000/10000 [==============================] - 0s 48us/step - loss: 101.7227
Epoch 182/3000
10000/10000 [==============================] - 0s 48us/step - loss: 101.6675
Epoch 183/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.7358
Epoch 184/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.7374
Epoch 185/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.7449
Epoch 186/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.7753
Epoch 187/3000
10000/10000 [==============================] - 0s 48us/step - loss: 101.7205
Epoch 188/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.5808
Epoch 189/3000
10000/10000 [==============================] - 0s 48us/step - loss: 101.5900
Epoch 190/3000
10000/10000 [==============================] - 1s 55us/step - loss: 101.7191
Epo

10000/10000 [==============================] - 0s 47us/step - loss: 101.3751
Epoch 270/3000
10000/10000 [==============================] - 0s 48us/step - loss: 101.2399
Epoch 271/3000
10000/10000 [==============================] - 0s 46us/step - loss: 101.3919
Epoch 272/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.3337
Epoch 273/3000
10000/10000 [==============================] - 0s 46us/step - loss: 101.4006
Epoch 274/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.3916
Epoch 275/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.3966
Epoch 276/3000
10000/10000 [==============================] - 0s 48us/step - loss: 101.4064
Epoch 277/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.3918
Epoch 278/3000
10000/10000 [==============================] - 0s 49us/step - loss: 101.3495
Epoch 279/3000
10000/10000 [==============================] - 0s 48us/step - loss: 101.3381
Epo

10000/10000 [==============================] - 0s 47us/step - loss: 101.1226
Epoch 359/3000
10000/10000 [==============================] - 0s 48us/step - loss: 101.2236
Epoch 360/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.1643
Epoch 361/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.2274
Epoch 362/3000
10000/10000 [==============================] - 0s 48us/step - loss: 101.1004
Epoch 363/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.1039
Epoch 364/3000
10000/10000 [==============================] - 0s 49us/step - loss: 101.1139
Epoch 365/3000
10000/10000 [==============================] - 0s 47us/step - loss: 101.2041
Epoch 366/3000
10000/10000 [==============================] - 1s 51us/step - loss: 101.1094
Epoch 367/3000
10000/10000 [==============================] - 1s 50us/step - loss: 101.2291
Epoch 368/3000
10000/10000 [==============================] - 1s 53us/step - loss: 101.1689
Epo

...

10000/10000 [==============================] - 0s 47us/step - loss: 100.4956
Epoch 2566/3000
10000/10000 [==============================] - 0s 49us/step - loss: 100.4668
Epoch 2567/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.5101
Epoch 2568/3000
10000/10000 [==============================] - 0s 50us/step - loss: 100.4756
Epoch 2569/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4538
Epoch 2570/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4549
Epoch 2571/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.5087
Epoch 2572/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.5662
Epoch 2573/3000
10000/10000 [==============================] - 0s 49us/step - loss: 100.4383
Epoch 2574/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4955
Epoch 2575/3000
10000/10000 [==============================] - 0s 47us/step - loss: 10

10000/10000 [==============================] - 0s 47us/step - loss: 100.4964
Epoch 2654/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4908
Epoch 2655/3000
10000/10000 [==============================] - 0s 46us/step - loss: 100.5057
Epoch 2656/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.5608
Epoch 2657/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4747
Epoch 2658/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.5354
Epoch 2659/3000
10000/10000 [==============================] - 0s 46us/step - loss: 100.4710
Epoch 2660/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.5321
Epoch 2661/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.5171
Epoch 2662/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.5085
Epoch 2663/3000
10000/10000 [==============================] - 0s 48us/step - loss: 10

10000/10000 [==============================] - 0s 48us/step - loss: 100.6073
Epoch 2742/3000
10000/10000 [==============================] - 0s 46us/step - loss: 100.4980
Epoch 2743/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.5465
Epoch 2744/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4882
Epoch 2745/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4726
Epoch 2746/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.4716
Epoch 2747/3000
10000/10000 [==============================] - 1s 50us/step - loss: 100.5353
Epoch 2748/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.5101
Epoch 2749/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.5612
Epoch 2750/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4720
Epoch 2751/3000
10000/10000 [==============================] - 0s 47us/step - loss: 10

10000/10000 [==============================] - 0s 48us/step - loss: 100.5116
Epoch 2830/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.4831
Epoch 2831/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.4802
Epoch 2832/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4842
Epoch 2833/3000
10000/10000 [==============================] - 0s 49us/step - loss: 100.5152
Epoch 2834/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.4664
Epoch 2835/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4520
Epoch 2836/3000
10000/10000 [==============================] - 0s 49us/step - loss: 100.5060
Epoch 2837/3000
10000/10000 [==============================] - 0s 49us/step - loss: 100.4045
Epoch 2838/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4626
Epoch 2839/3000
10000/10000 [==============================] - 0s 47us/step - loss: 10

10000/10000 [==============================] - 0s 48us/step - loss: 100.5613
Epoch 2918/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.5026
Epoch 2919/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4092
Epoch 2920/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.4573
Epoch 2921/3000
10000/10000 [==============================] - 0s 49us/step - loss: 100.4840
Epoch 2922/3000
10000/10000 [==============================] - 0s 49us/step - loss: 100.4351
Epoch 2923/3000
10000/10000 [==============================] - 0s 46us/step - loss: 100.4573
Epoch 2924/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.5031
Epoch 2925/3000
10000/10000 [==============================] - 0s 47us/step - loss: 100.4000
Epoch 2926/3000
10000/10000 [==============================] - 0s 48us/step - loss: 100.4011
Epoch 2927/3000
10000/10000 [==============================] - 0s 48us/step - loss: 10

AttributeError: 'list' object has no attribute 'shape'

In [149]:
estimator.fit(trainX, trainY, epochs=epochs,verbose=keras_verbose)


In [160]:
x=np.array([.5,.5,.5,.5,.5]).reshape(1,5)
print(10*np.sin(np.pi*x[:,0]*x[:,1]) - 20*(x[:,2]-0.05)**2 + 10*x[:,3] + 5*x[:,4]) 


[10.52106781]


In [35]:
x1=.5
x2=.5
x3=.5
x4=.5
x5=.5
print(10*np.sin(np.pi*x1*x2) - 20*(x3-0.05)**2 + 10*x4 + 5*x5 )


10.521067811865475


p

In [142]:
#numpy.random.seed(seed=None)

n_mc = 51
n_z = 3
k_n = 30
n=1000
# num_processes = 12
in_interval_2 = 0
in_interval_3 = 0
num_trials = 120
theta = nonlinear().theta()
start_time = datetime.now()
estimates = list()

for _ in range(0, num_trials):

    nz_average = list() 
    iv_pred = list()

    for i in range(0,n_z):
        fixed_point = random_sample.sample(1)    
        random_sample.drop(fixed_point.index)
        for _ in range(0,n_mc):
    #         with Pool(num_processes) as p:
    #             iv_pred.append(p.apply_async(estimate, 
    #                                          estimator, random_sample.sample(k_n).append(fixed_point)))
            iv_pred.append(estimate(estimator, random_sample.sample(k_n).append(fixed_point)))
        nz_average.append(np.mean(iv_pred[(i*n_mc):(i+1)*n_mc]))    
        random_sample.append(fixed_point) 

    zeta_1_k = np.var(nz_average)
    zeta_k_k = np.var(iv_pred)
    theta_k = np.mean(iv_pred)


    alpha = n / (n_mc*n_z)
    var_net = k_n**2 / alpha * zeta_1_k + zeta_k_k

   
 
    SE = (var_net / (n_mc * n_z))**(1/2)
   
    print("Zeta_1_K {}, Zeta_k_k {}, Theta_k {} SE {}".format(
        zeta_1_k, zeta_k_k, theta_k, SE))
    
    if(theta < theta_k + 2*SE  and theta > theta_k - 2*SE):
        in_interval_2 +=1
    if(theta < theta_k + 3*SE  and theta > theta_k - 3*SE):
        in_interval_3 +=1
    
    estimates.append((theta_k,SE))

print("This took {} seconds to run".format(datetime.now()-start_time))    
print("This is the coverage rate 95% {} 99.7% {}".format(
    in_interval_2 / num_trials, in_interval_3/num_trials))


Zeta_1_K 1.1576563119888306, Zeta_k_k 21.45486831665039, Theta_k 32.61699295043945 SE 1.087252766510343
Zeta_1_K 0.031117716804146767, Zeta_k_k 19.33538246154785, Theta_k 33.69559860229492 SE 0.3929134687078297
Zeta_1_K 0.005681868642568588, Zeta_k_k 22.582788467407227, Theta_k 33.732337951660156 SE 0.3907858830637161
Zeta_1_K 1.1675390005111694, Zeta_k_k 23.591114044189453, Theta_k 33.038448333740234 SE 1.0977137096395269
Zeta_1_K 0.05830392614006996, Zeta_k_k 15.775700569152832, Theta_k 34.081878662109375 SE 0.39443971348333257
Zeta_1_K 0.2346401810646057, Zeta_k_k 28.3934268951416, Theta_k 33.129520416259766 SE 0.6298842088202895
Zeta_1_K 0.4831989109516144, Zeta_k_k 23.871707916259766, Theta_k 34.00550842285156 SE 0.7687023185884434
Zeta_1_K 4.449669361114502, Zeta_k_k 29.502233505249023, Theta_k 34.232547760009766 SE 2.04878683141827
Zeta_1_K 2.0920002460479736, Zeta_k_k 23.283300399780273, Theta_k 33.801300048828125 SE 1.4265267804807131
Zeta_1_K 1.279849648475647, Zeta_k_k 26.26

Zeta_1_K 0.7372872829437256, Zeta_k_k 23.703134536743164, Theta_k 34.909637451171875 SE 0.9046993990281649
Zeta_1_K 1.609169840812683, Zeta_k_k 24.449417114257812, Theta_k 32.740699768066406 SE 1.2680902840844215
Zeta_1_K 0.6317813396453857, Zeta_k_k 30.540016174316406, Theta_k 33.31426239013672 SE 0.876476556750144
Zeta_1_K 0.2568223476409912, Zeta_k_k 24.8713321685791, Theta_k 33.778926849365234 SE 0.6274534560179841
Zeta_1_K 0.24484817683696747, Zeta_k_k 18.889497756958008, Theta_k 33.77042007446289 SE 0.5863651838288073
Zeta_1_K 0.008370445109903812, Zeta_k_k 24.07423210144043, Theta_k 33.645511627197266 SE 0.40605581245753675
Zeta_1_K 0.2350500077009201, Zeta_k_k 24.050168991088867, Theta_k 33.14414596557617 SE 0.6072360775937221
Zeta_1_K 0.8156879544258118, Zeta_k_k 21.86898422241211, Theta_k 33.92280960083008 SE 0.9365114502154349
Zeta_1_K 0.2012457698583603, Zeta_k_k 24.849008560180664, Theta_k 33.30803298950195 SE 0.5861168940811209
Zeta_1_K 0.0015861369902268052, Zeta_k_k 22.

In [126]:
theta = nonlinear().theta()
theta

33.41813569769473

In [143]:
stable_estimates_153_3000 = estimates.copy()
stable_estimates_153_3000

[(32.616993, 1.087252766510343),
 (33.6956, 0.3929134687078297),
 (33.732338, 0.3907858830637161),
 (33.03845, 1.0977137096395269),
 (34.08188, 0.39443971348333257),
 (33.12952, 0.6298842088202895),
 (34.00551, 0.7687023185884434),
 (34.232548, 2.04878683141827),
 (33.8013, 1.4265267804807131),
 (33.172985, 1.1504498898583817),
 (32.913944, 0.45533968031377997),
 (33.4081, 0.5523615131233287),
 (32.728, 0.5929344081499014),
 (33.736366, 1.1331169441851332),
 (32.29056, 0.474817715680134),
 (33.274227, 0.7258807451208761),
 (33.306995, 0.40625443545737555),
 (33.37333, 0.7988121079511362),
 (33.327026, 0.6474067930169103),
 (33.811314, 1.3894095320249107),
 (33.450207, 0.5676986365580399),
 (33.29663, 0.6162610736999818),
 (33.923702, 0.84112862617021),
 (32.80776, 0.516230706140393),
 (33.210907, 0.44986183833521326),
 (33.50256, 0.47266833950767817),
 (33.42406, 0.8121732934063762),
 (33.771206, 0.6259391177496292),
 (34.383904, 1.3901661252972235),
 (33.595066, 0.5110031736783451),
 

In [138]:
#stable_estimates.append(estimates)
stable_estimates_51_3000=stable_estimates_51_3000+estimates

In [144]:
with open('estimates_nonlinear_153_3000.csv', 'w') as f:
    writer = csv.writer(f , lineterminator='\n')
    for tup in stable_estimates_153_3000:
        writer.writerow(tup)

In [134]:
print("Zeta_1_K {}, Zeta_k_k {}, Theta_k {}".format(zeta_1_k, zeta_k_k, theta_k))

Zeta_1_K 0.7391712665557861, Zeta_k_k 18.45136260986328, Theta_k 34.3473014831543


In [65]:
fixed_point = random_sample.sample(1)
random_sample.drop(fixed_point.index)
print(fixed_point)   
print(random_sample.sample(1).append(fixed_point))   


            x         y
151  2.843675  3.636818
            x         y
403  8.277868  9.062615
151  2.843675  3.636818


# External Coverage Study

In [38]:


def external(estimator):    
    n_mc = 30
    n_z = 30
    k_n = 31
    m_n = 30
    n=1000
        
    random_sample = nonlinear().simulate(n)
    pred = list()
    
    
    for _ in range(m_n):
        pred.append(estimate(estimator, random_sample.sample(k_n)))  
    
    
    mc_pred = list()
    for _ in range(n_z):
        fixed_point = random_sample.sample(1)    
        random_sample.drop(fixed_point.index)
        iv_pred = list()
        for _ in range(0,n_mc):
            iv_pred.append(estimate(estimator, random_sample.sample(k_n-1).append(fixed_point)))
        random_sample.append(fixed_point) 
        mc_pred.append(np.mean(iv_pred))

    nz_pred = list()
    for _ in range(n_z):
        nz_pred.append(estimate(estimator, random_sample.sample(k_n)))  

    zeta_1_k = np.var(mc_pred)
    zeta_k_k = np.var(nz_pred)
    theta_k = np.mean(pred)


    alpha = n / (m_n)
    var_net = k_n**2 / alpha * zeta_1_k + zeta_k_k

   
 
    SE = (var_net / (m_n))**(1/2)
    print("Zeta_1_K {}, Zeta_k_k {}, Theta_k {} SE {}".format(
    zeta_1_k, zeta_k_k, theta_k, SE))
    return((theta_k, SE))

    
    
def coverage_prob(theta_k, SE, ci):
    theta = nonlinear().theta()
    
    if(theta < theta_k + 2*SE  and theta > theta_k - 2*SE):
        ci[0] +=1
    if(theta < theta_k + 3*SE  and theta > theta_k - 3*SE):
        ci[1] +=1
    
    return(ci)
    

    



n_mc = 30
n_z = 30
k_n = 31
m_n = 30
n=1000
# num_processes = 12
in_interval_2 = 0
in_interval_3 = 0
num_trials = 50
start_time = datetime.now()
estimates = list()

# estimator = build_linear_estimator()



# with Pool(8) as p:
#         print(p.map(external, [0]))
estimator = build_linear_estimator()
for _ in range(num_trials):       
    estimates.append(external(estimator))

print("This took {} seconds to run".format(datetime.now()-start_time))    
# print("This is the coverage rate 95% {} 99.7% {}".format(
#     in_interval_2 / num_trials, in_interval_3/num_trials))




Zeta_1_K 1.3307031393051147, Zeta_k_k 20.732728958129883, Theta_k 33.16799545288086 SE 1.4035300788166998
Zeta_1_K 1.8167887926101685, Zeta_k_k 13.05528450012207, Theta_k 34.075157165527344 SE 1.4768582124572558
Zeta_1_K 0.890783965587616, Zeta_k_k 18.113676071166992, Theta_k 32.595157623291016 SE 1.2082353219890287
Zeta_1_K 0.6629222631454468, Zeta_k_k 21.30449867248535, Theta_k 31.813154220581055 SE 1.1606973122361772
Zeta_1_K 1.2972795963287354, Zeta_k_k 17.31553840637207, Theta_k 33.58317565917969 SE 1.3505074252360296
Zeta_1_K 1.5637774467468262, Zeta_k_k 25.45545196533203, Theta_k 33.732391357421875 SE 1.533396619219818
Zeta_1_K 1.290860652923584, Zeta_k_k 16.106128692626953, Theta_k 33.14133834838867 SE 1.3331871751110054
Zeta_1_K 0.7285254001617432, Zeta_k_k 15.268172264099121, Theta_k 33.852535247802734 SE 1.099568999665508
Zeta_1_K 1.1787950992584229, Zeta_k_k 19.899606704711914, Theta_k 35.43388748168945 SE 1.3402023406477626
Zeta_1_K 1.3933953046798706, Zeta_k_k 17.17437744

In [39]:
with open('nonlinear_3000_external_30_30_50.csv', 'w') as f:
    writer = csv.writer(f , lineterminator='\n')
    for tup in estimates:
        writer.writerow(tup)


In [40]:
ci=[0,0]  
for i in range(len(estimates)):
    ci = coverage_prob(estimates[i][0], estimates[i][1], ci)
print("Coverage rate, CI2 {} CI3{}".format(ci[0]/50, ci[1]/50))

Coverage rate, CI2 0.92 CI31.0


# Linear External Coverage Study 1:
n_mc = 30
n_z = 3
k_n = 31
m_n = 30
n=1000
num_lay_nodes = 10
lr = .01
epochs = 1000
keras_verbose = 0
random_sample = linear().simulate(1000)
num_trials = 150



Zeta_k_k 32.042850494384766 N_Z 30
                            

# Linear External Coverage Study 2:
n_mc = 30
n_z = 30
k_n = 31
m_n = 30
n=1000
num_lay_nodes = 10
lr = .01
epochs = 1000
keras_verbose = 0
random_sample = linear().simulate(1000)
num_trials = 50

Results .96 CI2 and .98 CI3

# Nonlinear External Coverage Study 1:
n_mc = 30
n_z = 30
k_n = 31
m_n = 30
n=1000
num_lay_nodes = 10
lr = .01
epochs = 3000
keras_verbose = 0
random_sample = linear().simulate(1000)
num_trials = 50

Results .92 CI2 and 1 CI3